# DCR Extension Toturial

This file will give a walkthrough of all the extension that has been made for the pm4py library
Namely it will go through how to create a DCR Graph, this can be done manually or automatically by the implemented DisCoveR algorithm. Further it will showcase how conformance checking can be used to determine fitness.


## Creating DCR Graphs
First lets take a look at how to create a basic DCR Graph manually, note that the underlying attribute in the dcr graph objects consists of set and dictionaries, so one can access them through the property, and call them as such:

In [43]:
from pm4py.objects.dcr.obj import DcrGraph
graph = DcrGraph()
graph.events.add("A")
graph.events.add("B")
graph.events.add("C")
graph.labels.add("activity1")
graph.labels.add("activity2")
graph.labels.add("activity3")
graph.label_mapping["activity1"] = {"A"}
graph.label_mapping["activity2"] = {"B"}
graph.label_mapping["activity3"] = {"C"}
graph.conditions["A"] = set("B")
graph.conditions["B"] = set("C")
graph.responses["A"] = set("C")
graph.excludes["C"] = set("B")
graph.conditions["A"].add("D")
graph.includes["A"] = set("C")
graph.marking.included.add("A")
graph.marking.included.add("B")
graph.marking.included.add("C")
graph.marking.included.add("D")
print(graph)

events: {'C', 'B', 'A'}
marking: {executed: set(), included: {'C', 'B', 'D', 'A'}, pending: set()}
labels: {'activity2', 'activity3', 'activity1'}
conditionsFor: {'A': {'B', 'D'}, 'B': {'C'}}
responseTo: {'A': {'C'}}
includesTo: {'A': {'C'}}
excludesTo: {'C': {'B'}}
labelMapping: {'activity1': {'A'}, 'activity2': {'B'}, 'activity3': {'C'}}


When the graph has been constructued, one can get access to the following commands, to get the different properties of the dcr graph, such as the size define by the number of constraints, the events associated with the activity, or vice versa.

In [36]:
print(graph.get_constraints())
print(graph.get_activity("A"))
print(graph.get_event("activity1"))
del graph

6
activity1
A


If one calls get_event() or get_activity() with a value that doesn't exists, it will return the input value, this was implemted for the conformance checking, as if someones tries to get the labelmapping, but the the event doesn't exist the test would be interrupted, this way if it doesn't exists it will be noted by the conformance tools, and be used in conformance result.

Now to discover a model, with a given input log, a simplified interface has been created, such that the implemented algorithm are easy to get access to, and therefore allow, for easier use

In [44]:
import pm4py

log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log)
print(graph)
del graph

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

events: {'examine thoroughly', 'register request', 'reject request', 'check ticket', 'reinitiate request', 'examine casually', 'pay compensation', 'decide'}
marking: {executed: set(), included: {'examine thoroughly', 'register request', 'reject request', 'check ticket', 'reinitiate request', 'examine casually', 'pay compensation', 'decide'}, pending: set()}
labels: {'examine thoroughly', 'reject request', 'register request', 'check ticket', 'reinitiate request', 'examine casually', 'pay compensation', 'decide'}
conditionsFor: {'reinitiate request': {'decide'}, 'pay compensation': {'decide'}, 'reject request': {'decide'}, 'decide': {'check ticket', 'examine casually'}, 'examine thoroughly': {'register request'}, 'check ticket': {'register request'}, 'examine casually': {'register request'}}
responseTo: {'register request': {'check ticket'}, 'reinitiate request': {'check ticket'}, 'examine thoroughly': {'decide'}, 'check ticket': {'decide'}, 'examine casually': {'decide'}}
includesTo: {'

As according to the rest of the library, the simplified interface, takes in extra values for which one can use to specify the naming convention in the attribute, such that it can mine the log without failure.

Additionally, the discover miner has been extended to allow for mining of roles, we will use the log of the running example again

In [45]:
graph, _ = pm4py.discover_dcr(log,process_type={"roles"},group_key="org:resource")
print(graph)
del graph

events: {'examine thoroughly', 'register request', 'reject request', 'check ticket', 'reinitiate request', 'examine casually', 'pay compensation', 'decide'}
marking: {executed: set(), included: {'examine thoroughly', 'register request', 'reject request', 'check ticket', 'reinitiate request', 'examine casually', 'pay compensation', 'decide'}, pending: set()}
labels: {'examine thoroughly', 'reject request', 'register request', 'check ticket', 'reinitiate request', 'examine casually', 'pay compensation', 'decide'}
conditionsFor: {'reinitiate request': {'decide'}, 'pay compensation': {'decide'}, 'reject request': {'decide'}, 'decide': {'check ticket', 'examine casually'}, 'examine thoroughly': {'register request'}, 'check ticket': {'register request'}, 'examine casually': {'register request'}}
responseTo: {'register request': {'check ticket'}, 'reinitiate request': {'check ticket'}, 'examine thoroughly': {'decide'}, 'check ticket': {'decide'}, 'examine casually': {'decide'}}
includesTo: {'

If one doesn't wish to use the simplified interface, there is a more indirect way of performing the process mining technique

In [5]:
from pm4py.algo.discovery.dcr_discover.variants.dcr_discover import Discover
from pm4py.objects.dcr.obj import DcrGraph
disc = Discover()
graph = DcrGraph(disc.mine(log)[0])
del log

Note that the discover miner also returns the abstraction log used for mining the DCR Graph, therefore it is needed to perform to specify the first iterative of the tuple.

## Importing/Exporting DCR Graphs

## conformance checking
Two different technique for checking conformance for a dcr graph has been implemented
### Rule Checking
The first technique is the, a quite straight forward approach, is to take an event log, and mine based on the constraints within the graph. The first technique takes in a whole log and produces an output, of a a list of dictioneries of the values associated with the mining:

In [48]:
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log)
conf_res = pm4py.conformance_dcr(log, graph)
print(conf_res)
del conf_res
del graph
del log

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

[{'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}, {'no_constr_total': 30, 'deviations': [], 'no_dev_total': 0, 'dev_fitness': 1.0, 'is_fit': True}]


Due to the DisCoveR miners property of always producing a Graph with a perfect fitness, the output will therefore have no deviations. Additionally to allow mining of DCR Graphs, it also possible to check for deviations in role assignment.

In [49]:
# Given an event log and discovering a dcr
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log, process_type={'roles'}, group_key="org:resource")

# when the roles are changed and conformance is performed
log = log.replace("Mike", "Brenda")
conf_res = pm4py.conformance_dcr(log, graph, group_key="org:resource")
print(conf_res)

parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

[{'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}]


If one wishes, one can return the values as a pandas dataframe


In [41]:
log = pm4py.read_xes("../tests/input_data/running-example.xes")
graph, _ = pm4py.discover_dcr(log, process_type={'roles'}, group_key="org:resource")

# when the roles are changed and conformance is performed
log = log.replace("Mike", "Brenda")
conf_res = pm4py.conformance_dcr(log, graph, group_key="org:resource",return_diagnostics_dataframe=True)



parsing log, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\jonas\DCR4Py\pm4py-dcr\pm4py\utils.py:542: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


Just as the previous example one can the under lying algorithm directly if they wishes to do so, in this case one can call the class used for conformance checking:

In [42]:
from pm4py.algo.conformance.dcr.variants.classic import RuleBasedConformance
parameters = pm4py.utils.get_properties(log,group_key="org:resource")
rulecheck = RuleBasedConformance(log, graph,parameters)
conf_res = rulecheck.apply_conformance()
print(conf_res)

[{'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}, {'no_constr_total': 49, 'deviations': [('roleViolation', 'Brenda')], 'no_dev_total': 1, 'dev_fitness': 0.9795918367346939, 'is_fit': False}]


### alignment

The extensions also allow for the determining the optimal alignment of a DCR Graph